# **Prompt Generation**

In this notebook is illustrated how to generate prompts by using the implemented libraries.

Note that these prompts are essentially designed for GPT models, or for those supporting the following roles:
- **system**,  which provides high-level instructions to guide model’s behavior throughout the conversation;
- **user**, it is the model’s response, usually simulated when using few-shot prompting;
- **assistant**, that presents queries related to the task the LLM is asked to perform.

We designed different versions of prompts for the same task. Then, all of them are shown within this notebook.
Such variants are briefly described below:
- *v1.0*, the model is asked to generate the whole questionnaire in a single shot, directly in the JSON format.
- *v1.1*, the model is asked to generate the whole questionnaire in a single shot, but it's asked to explain firstly the content of it (kinda reasoning) and then format it as a JSON. Moreover, the Chain-of-Thought is integrated to let the model "think" better about the structure of the questionnaire.
- *v2.0*, the generation task is split into two parts: firstly, the model is aked to generate the questionnaire without any format constraint, then (in another call), the model is asked to simply convert it into a JSON.
- *v2.1*, the conversation follows the previous verion's flow, but it integrates the Chain-of-Thought technique to let the model think deeply about the content of the questionnaire.

In [1]:
import os
import sys
sys.path.append('\\'.join(os.getcwd().split('\\')[:-1])+'\\src')

from src.data.TFQuestionnairesDataset import TFQuestionnairesDataset
from src.prompts.PredictionSystemPrompt import PredictionSystemPrompt
from src.prompts.PredictionUserPrompt import PredictionUserPrompt
from src.prompts.PredictionAssistantPrompt import PredictionAssistantPrompt

from pprint import pprint

## **Survey questionnaire generation**

The first step of this thesis work is the questionnaire generaion. The designed prompts are reported below.

In [2]:
# ----------------
# Load data
# ----------------
dataset = TFQuestionnairesDataset()
dataset.load_data(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

samples = dataset.get_sample_questionnaire_data()
questionnaire_id = samples.questionnaires["ID"]

question_types = samples.get_question_types()

### *System prompt*

#### v1.0

The **system** prompt has two variants according to the given specifications:
1. with all parameters: topic, number and type of questions to be generated
2. with only the questionnaire topic

Both of them are structured as follows:
1. **Role definition**, used to define the role the LLM has to impersonate.
2. **Input definition**, which define the information that the user can specify.
3. **Task definition**, which is used to instruct the LLM on the task it has to perform.
4. **Output definition**, it constrates the LLM to use the specified format while generating its response.
5. **Question types**, used to define the admissible question types.
6. **Style command**, used to stimulate the model to be more syntactically variable.

In [3]:
system = PredictionSystemPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

full_params = system.build_prompt(has_full_params=True, qst_types_df=dataset.question_types)
only_topic = system.build_prompt(has_full_params=False, qst_types_df=dataset.question_types)

In [4]:
pprint(full_params)

('You are a Questionnaire Generator in the Human Resource Management field.\n'
 'The user will ask you to generate a questionnaire specifying the topic and '
 'the number of questions.\n'
 'If the user does not specify a valid topic, reply with "Sorry I cant help '
 'you".\n'
 'If the topic is valid, reply with only a JSON, which must respect the '
 'following format:\n'
 ' - The root of the JSON is an object that contains a single property '
 "'data'.\n"
 "        - The 'data' property is an object that contains a single property "
 "'TF_QUESTIONNAIRES'.\n"
 "        - 'TF_QUESTIONNAIRES' is an array of only one element, which "
 'represents a questionnaire. It has the following properties:\n'
 "            - 'CODE': (string) the questionnaire's code.\n"
 "            - 'NAME': (string) the questionnaire's name.\n"
 "            - 'TYPE_ID': (int) which is equal to 3.\n"
 "            - '_TF_QUESTIONS': An array of objects, each representing a "
 'question.\n'
 "        - Each questio

In [5]:
pprint(only_topic)

('You are a Questionnaire Generator in the Human Resource Management field.\n'
 'The user will ask you to generate a questionnaire about a specified topic.\n'
 'If the user does not specify a valid topic, reply with "Sorry I cant help '
 'you".\n'
 'If the topic is valid, reply with only a JSON, which must respect the '
 'following format:\n'
 ' - The root of the JSON is an object that contains a single property '
 "'data'.\n"
 "        - The 'data' property is an object that contains a single property "
 "'TF_QUESTIONNAIRES'.\n"
 "        - 'TF_QUESTIONNAIRES' is an array of only one element, which "
 'represents a questionnaire. It has the following properties:\n'
 "            - 'CODE': (string) the questionnaire's code.\n"
 "            - 'NAME': (string) the questionnaire's name.\n"
 "            - 'TYPE_ID': (int) which is equal to 3.\n"
 "            - '_TF_QUESTIONS': An array of objects, each representing a "
 'question.\n'
 "        - Each question in the '_TF_QUESTIONS' arra

#### v1.1

The **system** prompt has two variants according to the given specifications:
1. with all parameters: topic, number and type of questions to be generated
2. with only the questionnaire topic

Both of them are structured as follows:
1. **Role definition**, used to define the role the LLM has to impersonate.
2. **Input definition**, which define the information that the user can specify.
3. **Task definition**, which is used to instruct the LLM on the task it has to perform.
4. **Output definition**, it constrates the LLM to use the specified format while generating its response.
5. **Question types**, used to define the admissible question types.
6. **Style command**, used to stimulate the model to be more syntactically variable.
7. **Chain of Thought**, used to induce the model thinking deeply about the content of the questionnaire.

In [6]:
system = PredictionSystemPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="1.1")

full_params = system.build_prompt(has_full_params=True, qst_types_df=dataset.question_types)
only_topic = system.build_prompt(has_full_params=False, qst_types_df=dataset.question_types)

In [7]:
pprint(full_params)

('You are a Questionnaire Generator in the Human Resource Management field.\n'
 'The user will ask you to generate a questionnaire specifying the topic and '
 'the number of questions.\n'
 'Firstly describe how the questionnaire is structured. Start with '
 "'Explanation: '. Then format the questionnaire as a JSON, which must respect "
 'the following format:\n'
 ' - The root of the JSON is an object that contains a single property '
 "'data'.\n"
 "        - The 'data' property is an object that contains a single property "
 "'TF_QUESTIONNAIRES'.\n"
 "        - 'TF_QUESTIONNAIRES' is an array of only one element, which "
 'represents a questionnaire. It has the following properties:\n'
 "            - 'CODE': (string) the questionnaire's code.\n"
 "            - 'NAME': (string) the questionnaire's name.\n"
 "            - 'TYPE_ID': (int) which is equal to 3.\n"
 "            - '_TF_QUESTIONS': An array of objects, each representing a "
 'question.\n'
 "        - Each question in the 

In [8]:
pprint(only_topic)

('You are a Questionnaire Generator in the Human Resource Management field.\n'
 'The user will ask you to generate a questionnaire about a specified topic.\n'
 'Firstly describe how the questionnaire is structured. Start with '
 "'Explanation: '. Then format the questionnaire as a JSON, which must respect "
 'the following format:\n'
 ' - The root of the JSON is an object that contains a single property '
 "'data'.\n"
 "        - The 'data' property is an object that contains a single property "
 "'TF_QUESTIONNAIRES'.\n"
 "        - 'TF_QUESTIONNAIRES' is an array of only one element, which "
 'represents a questionnaire. It has the following properties:\n'
 "            - 'CODE': (string) the questionnaire's code.\n"
 "            - 'NAME': (string) the questionnaire's name.\n"
 "            - 'TYPE_ID': (int) which is equal to 3.\n"
 "            - '_TF_QUESTIONS': An array of objects, each representing a "
 'question.\n'
 "        - Each question in the '_TF_QUESTIONS' array has the

#### v2.0

The **system** prompt has two variants according to the given specifications:
1. with all parameters: topic, number and type of questions to be generated
2. with only the questionnaire topic

Both of them are structured as follows:
1. **Role definition**, used to define the role the LLM has to impersonate.
2. **Input definition**, which define the information that the user can specify.
3. **Task definition**, which is used to instruct the LLM on the task it has to perform.
4. **Style command**, used to stimulate the model to be more syntactically variable.

In [9]:
system = PredictionSystemPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="2.0")

full_params = system.build_prompt(has_full_params=True, qst_types_df=dataset.question_types)
only_topic = system.build_prompt(has_full_params=False, qst_types_df=dataset.question_types)

In [10]:
pprint(full_params)

('You are a Questionnaire Generator that helps HR users investigate phenomena '
 'within the company.\n'
 'Firstly, the user will ask you to generate a questionnaire specifying the '
 'topic and the number of questions.\n'
 'Then, the user will ask to convert it into a specified format.\n'
 'Be creative and vary the syntax of your questions to enhance user '
 'engagement. \n')


In [11]:
pprint(only_topic)

('You are a Questionnaire Generator that helps HR users investigate phenomena '
 'within the company.\n'
 'Firstly, the user will ask you to generate a questionnaire specifying the '
 'topic.\n'
 'Then, the user will ask to convert it into a specified format.\n'
 'Be creative and vary the syntax of your questions to enhance user '
 'engagement. \n')


#### v2.1

The **system** prompt has two variants according to the given specifications:
1. with all parameters: topic, number and type of questions to be generated
2. with only the questionnaire topic

Both of them are structured as follows:
1. **Role definition**, used to define the role the LLM has to impersonate.
2. **Input definition**, which define the information that the user can specify.
3. **Task definition**, which is used to instruct the LLM on the task it has to perform.
4. **Style command**, used to stimulate the model to be more syntactically variable.
5. **Chain of Thought**, used to stimulate a better organization and quality of the questionnaire content.

In [12]:
system = PredictionSystemPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="2.1")

full_params = system.build_prompt(has_full_params=True, qst_types_df=dataset.question_types)
only_topic = system.build_prompt(has_full_params=False, qst_types_df=dataset.question_types)

In [13]:
pprint(full_params)

('You are a Questionnaire Generator that helps HR users in investigating '
 'phenomena within the company.\n'
 'Firstly, the user will ask you to generate a questionnaire specifying the '
 'topic and the number of quesions.\n'
 'Then, the user will ask to convert it in a specified format.\n'
 'Be creative and vary the syntax of your questions to enhance user '
 'engagement. \n'
 'Think about the questionnaire content step by step.\n')


In [14]:
pprint(only_topic)

('You are a Questionnaire Generator that helps HR users in investigating '
 'phenomena within the company.\n'
 'Firstly, the user will ask you to generate a questionnaire specifying the '
 'topic.\n'
 'Then, the user will ask to convert it in a specified format.\n'
 'Be creative and vary the syntax of your questions to enhance user '
 'engagement. \n'
 'Think about the questionnaire content step by step.\n')


### *Assistant prompt*

The **assistant** prompt represents the LLM's output. It's used only when using the *few-shot* prompting technique. Note that is doesn't change according to the parameters specified by the user.

#### v1.0

In this case, it simply corresponds to the JSON of the questionnaire to be generated.

In [15]:
assistant = PredictionAssistantPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

formatted_json = dataset.to_json(questionnaire_id)

response = assistant.build_prompt(params=[formatted_json])
pprint(response)

('{"data": {"TF_QUESTIONNAIRES": [{"CODE": "Stress Survey", "NAME": "Stress '
 'Survey", "_TF_QUESTIONS": [{"CODE": "1", "NAME": "I often feel overwhelmed '
 'by tasks which I\'m not sure to complete in time", "TYPE_ID": 3, '
 '"DISPLAY_ORDER": 1, "_TF_ANSWERS": [{"ANSWER": "Completely Disagree"}, '
 '{"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": "5"}, '
 '{"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}]}, {"CODE": "2", "NAME": "I\'m not comfortable '
 'with my working Time schedule and I would like to shift or reduce it", '
 '"TYPE_ID": 3, "DISPLAY_ORDER": 2, "_TF_ANSWERS": [{"ANSWER": "Completely '
 'Disagree"}, {"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": '
 '"5"}, {"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}]}, {"CODE": "3", "NAME": "I\'m often frustrated '
 'because I\'m allocated on tasks which I\'m not really interested to", '
 '"TYPE_ID":

#### v1.1

It is structured as follows:
1. **Explanation**, given that there's no deep content description of the questionnaire in the dataset, we put a placeholder.
2. **JSON**, the questionnaire formatted as a JSON.

In [16]:
assistant = PredictionAssistantPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="1.1")

formatted_json = dataset.to_json(questionnaire_id)

response = assistant.build_prompt(params=[formatted_json])
pprint(response)

('Explanation: <your_explanation>\n'
 '\n'
 '{"data": {"TF_QUESTIONNAIRES": [{"CODE": "Stress Survey", "NAME": "Stress '
 'Survey", "_TF_QUESTIONS": [{"CODE": "1", "NAME": "I often feel overwhelmed '
 'by tasks which I\'m not sure to complete in time", "TYPE_ID": 3, '
 '"DISPLAY_ORDER": 1, "_TF_ANSWERS": [{"ANSWER": "Completely Disagree"}, '
 '{"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": "5"}, '
 '{"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}]}, {"CODE": "2", "NAME": "I\'m not comfortable '
 'with my working Time schedule and I would like to shift or reduce it", '
 '"TYPE_ID": 3, "DISPLAY_ORDER": 2, "_TF_ANSWERS": [{"ANSWER": "Completely '
 'Disagree"}, {"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": '
 '"5"}, {"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}]}, {"CODE": "3", "NAME": "I\'m often frustrated '
 'because I\'m allocated on tasks which I\'

#### v2.0

In this scenario, we modeled the questionnaire generation in a conversation made up of two steps as initially stated. 

Then the first **assistant** prompt corresponds to the questionnaire content in an unstructured format. TODO: how do we do this?
The second one is the corresponding JSON.

In [17]:
assistant = PredictionAssistantPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="2.0")

free_text = dataset.to_text(questionnaire_id)
formatted_json = dataset.to_json(questionnaire_id)

content = assistant.build_prompt(params=[free_text])
converted = assistant.build_prompt(params=[formatted_json])

In [18]:
pprint(content)

("**I often feel overwhelmed by tasks which I'm not sure to complete in "
 'time**\n'
 '  - Completely Disagree\n'
 '  - 2\n'
 '  - 3\n'
 '  - 4\n'
 '  - 5\n'
 '  - 6\n'
 '  - 7\n'
 '  - 8\n'
 '  - 9\n'
 '  - Strongly Agree\n'
 '\n'
 "**I'm not comfortable with my working Time schedule and I would like to "
 'shift or reduce it**\n'
 '  - Completely Disagree\n'
 '  - 2\n'
 '  - 3\n'
 '  - 4\n'
 '  - 5\n'
 '  - 6\n'
 '  - 7\n'
 '  - 8\n'
 '  - 9\n'
 '  - Strongly Agree\n'
 '\n'
 "**I'm often frustrated because I'm allocated on tasks which I'm not really "
 'interested to**\n'
 '  - Completely Disagree\n'
 '  - 2\n'
 '  - 3\n'
 '  - 4\n'
 '  - 5\n'
 '  - 6\n'
 '  - 7\n'
 '  - 8\n'
 '  - 9\n'
 '  - Strongly Agree\n'
 '\n'
 "**I often work overtime and I don't have time for myself**\n"
 '  - Completely Disagree\n'
 '  - 2\n'
 '  - 3\n'
 '  - 4\n'
 '  - 5\n'
 '  - 6\n'
 '  - 7\n'
 '  - 8\n'
 '  - 9\n'
 '  - Strongly Agree\n'
 '\n'
 '**The place where I use to work for most of my time is not

In [19]:
pprint(converted)

('{"data": {"TF_QUESTIONNAIRES": [{"CODE": "Stress Survey", "NAME": "Stress '
 'Survey", "_TF_QUESTIONS": [{"CODE": "1", "NAME": "I often feel overwhelmed '
 'by tasks which I\'m not sure to complete in time", "TYPE_ID": 3, '
 '"DISPLAY_ORDER": 1, "_TF_ANSWERS": [{"ANSWER": "Completely Disagree"}, '
 '{"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": "5"}, '
 '{"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}]}, {"CODE": "2", "NAME": "I\'m not comfortable '
 'with my working Time schedule and I would like to shift or reduce it", '
 '"TYPE_ID": 3, "DISPLAY_ORDER": 2, "_TF_ANSWERS": [{"ANSWER": "Completely '
 'Disagree"}, {"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": '
 '"5"}, {"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}]}, {"CODE": "3", "NAME": "I\'m often frustrated '
 'because I\'m allocated on tasks which I\'m not really interested to", '
 '"TYPE_ID":

#### v2.1

In this scenario, we modeled the questionnaire generation in a conversation made up of two steps as initially stated. 

Then the first **assistant** prompt corresponds to the questionnaire content in an unstructured format. TODO: how do we do this?
The second one is the corresponding JSON.

In [20]:
assistant = PredictionAssistantPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="2.1")

free_text = dataset.to_text(questionnaire_id)
formatted_json = dataset.to_json(questionnaire_id)

content = assistant.build_prompt(params=[free_text])
converted = assistant.build_prompt(params=[formatted_json])

In [21]:
pprint(content)

("**I often feel overwhelmed by tasks which I'm not sure to complete in "
 'time**\n'
 '  - Completely Disagree\n'
 '  - 2\n'
 '  - 3\n'
 '  - 4\n'
 '  - 5\n'
 '  - 6\n'
 '  - 7\n'
 '  - 8\n'
 '  - 9\n'
 '  - Strongly Agree\n'
 '\n'
 "**I'm not comfortable with my working Time schedule and I would like to "
 'shift or reduce it**\n'
 '  - Completely Disagree\n'
 '  - 2\n'
 '  - 3\n'
 '  - 4\n'
 '  - 5\n'
 '  - 6\n'
 '  - 7\n'
 '  - 8\n'
 '  - 9\n'
 '  - Strongly Agree\n'
 '\n'
 "**I'm often frustrated because I'm allocated on tasks which I'm not really "
 'interested to**\n'
 '  - Completely Disagree\n'
 '  - 2\n'
 '  - 3\n'
 '  - 4\n'
 '  - 5\n'
 '  - 6\n'
 '  - 7\n'
 '  - 8\n'
 '  - 9\n'
 '  - Strongly Agree\n'
 '\n'
 "**I often work overtime and I don't have time for myself**\n"
 '  - Completely Disagree\n'
 '  - 2\n'
 '  - 3\n'
 '  - 4\n'
 '  - 5\n'
 '  - 6\n'
 '  - 7\n'
 '  - 8\n'
 '  - 9\n'
 '  - Strongly Agree\n'
 '\n'
 '**The place where I use to work for most of my time is not

In [22]:
pprint(converted)

('{"data": {"TF_QUESTIONNAIRES": [{"CODE": "Stress Survey", "NAME": "Stress '
 'Survey", "_TF_QUESTIONS": [{"CODE": "1", "NAME": "I often feel overwhelmed '
 'by tasks which I\'m not sure to complete in time", "TYPE_ID": 3, '
 '"DISPLAY_ORDER": 1, "_TF_ANSWERS": [{"ANSWER": "Completely Disagree"}, '
 '{"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": "5"}, '
 '{"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}]}, {"CODE": "2", "NAME": "I\'m not comfortable '
 'with my working Time schedule and I would like to shift or reduce it", '
 '"TYPE_ID": 3, "DISPLAY_ORDER": 2, "_TF_ANSWERS": [{"ANSWER": "Completely '
 'Disagree"}, {"ANSWER": "2"}, {"ANSWER": "3"}, {"ANSWER": "4"}, {"ANSWER": '
 '"5"}, {"ANSWER": "6"}, {"ANSWER": "7"}, {"ANSWER": "8"}, {"ANSWER": "9"}, '
 '{"ANSWER": "Strongly Agree"}]}, {"CODE": "3", "NAME": "I\'m often frustrated '
 'because I\'m allocated on tasks which I\'m not really interested to", '
 '"TYPE_ID":

### *User prompt*

#### v1.0

It has to variants according to the number of parameters specified by the user. 

The first parameter corresponds to the questionnaire topic, and the second one to the question number (if needed).

In [23]:
user = PredictionUserPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="1.0")

full_params = user.build_prompt(has_full_params=True, params=["career satisfaction", 10])
only_topic = user.build_prompt(has_full_params=False, params=["career satisfaction"])

In [24]:
pprint(full_params)

'Generate me a questionnaire on career satisfaction with 10 questions\n'


In [25]:
pprint(only_topic)

'Generate me a questionnaire on career satisfaction\n'


#### v1.1

There are no changes with respect to the previous version.

In [26]:
user = PredictionUserPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="1.1")

full_params = user.build_prompt(has_full_params=True, params=["career satisfaction", 10])
only_topic = user.build_prompt(has_full_params=False, params=["career satisfaction"])

In [27]:
pprint(full_params)

'Generate me a questionnaire on career satisfaction with 10 questions\n'


In [28]:
pprint(only_topic)

'Generate me a questionnaire on career satisfaction\n'


#### v2.0

Similarly to what done for the roles above, the user prompt varies according to the "current step in the conversation".

At the fist iteration (step), the user will ask to generate the questionnaire without specifying any format. In this case there's no difference wrt the previous versions, so it changes according to the specified parameters.

Instead, the next step aims to convert the generated questinnaire to a JSON. Therefore the prompt includes the description of the JSON structure and the available question types.

In [29]:
user = PredictionUserPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="2.0")

full_params = user.build_prompt(has_full_params=True, params=["career satisfaction", 10])
only_topic = user.build_prompt(has_full_params=False, params=["career satisfaction"])
convert = user.build_prompt(has_full_params=False, qst_types_df=dataset.question_types, prompt_task="CONVERT")

In [30]:
pprint(full_params)

'Generate me a questionnaire on career satisfaction with 10 questions\n'


In [31]:
pprint(only_topic)

'Generate me a questionnaire on career satisfaction\n'


In [32]:
pprint(convert)

('Convert it to a JSON which must respect the following format:\n'
 ' - The root of the JSON is an object that contains a single property '
 "'data'.\n"
 "        - The 'data' property is an object that contains a single property "
 "'TF_QUESTIONNAIRES'.\n"
 "        - 'TF_QUESTIONNAIRES' is an array of only one element, which "
 'represents a questionnaire. It has the following properties:\n'
 "            - 'CODE': (string) the questionnaire's code.\n"
 "            - 'NAME': (string) the questionnaire's name.\n"
 "            - 'TYPE_ID': (int) which is equal to 3.\n"
 "            - '_TF_QUESTIONS': An array of objects, each representing a "
 'question.\n'
 "        - Each question in the '_TF_QUESTIONS' array has the following "
 'properties:\n'
 "            - 'CODE': (string) the question's unique code.\n"
 "            - 'NAME': (string) the question's content.\n"
 "            - 'TYPE_ID': (int) the question's type.\n"
 "            - 'DISPLAY_ORDER': (int) the question's disp

#### v2.1

Similarly to what done for the roles above, the user prompt varies according to the "current step in the conversation".

At the fist iteration (step), the user will ask to generate the questionnaire without specifying any format. In this case there's no difference wrt the previous versions, so it changes according to the specified parameters.

Instead, the next step aims to convert the generated questinnaire to a JSON. Therefore the prompt includes the description of the JSON structure and the available question types.

In [33]:
user = PredictionUserPrompt(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)), prompt_version="2.1")

full_params = user.build_prompt(has_full_params=True, params=["career satisfaction", 10])
only_topic = user.build_prompt(has_full_params=False, params=["career satisfaction"])
convert = user.build_prompt(has_full_params=False, qst_types_df=dataset.question_types, prompt_task="CONVERT")

In [34]:
pprint(full_params)

'Generate me a questionnaire on career satisfaction with 10 questions\n'


In [35]:
pprint(only_topic)

'Generate me a questionnaire on career satisfaction\n'


In [36]:
pprint(convert)

('Convert it to a JSON which must respect the following format:\n'
 ' - The root of the JSON is an object that contains a single property '
 "'data'.\n"
 "        - The 'data' property is an object that contains a single property "
 "'TF_QUESTIONNAIRES'.\n"
 "        - 'TF_QUESTIONNAIRES' is an array of only one element, which "
 'represents a questionnaire. It has the following properties:\n'
 "            - 'CODE': (string) the questionnaire's code.\n"
 "            - 'NAME': (string) the questionnaire's name.\n"
 "            - 'TYPE_ID': (int) which is equal to 3.\n"
 "            - '_TF_QUESTIONS': An array of objects, each representing a "
 'question.\n'
 "        - Each question in the '_TF_QUESTIONS' array has the following "
 'properties:\n'
 "            - 'CODE': (string) the question's unique code.\n"
 "            - 'NAME': (string) the question's content.\n"
 "            - 'TYPE_ID': (int) the question's type.\n"
 "            - 'DISPLAY_ORDER': (int) the question's disp